In [ ]:
import cv2
import numpy as np
from math import sqrt
from matplotlib import pyplot as plt
import sys
sys.path.append('../')
from utils.utils import read_from_env_file

env_vars = read_from_env_file()

%matplotlib inline

In [ ]:
%ll ../../data

In [ ]:
def plot_image(img, figsize=(24,16)):
    plt.close('all')
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(img, cmap='gray')
    #ax.axis('off')
    plt.show()

In [ ]:
def extract_frames_from_video(path, frames_steps=10):
    """ Returns a list of frames of input video """
    # Open video and get number of frames
    cap = cv2.VideoCapture(path)
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f'Frames in video: {frames}')
    
    # Frames to extract
    frames_list = [int(frames/frames_steps)*step
                   for step in range(frames_steps)]

    # Extract frames
    frames = list()
    for frame_n in frames_list:
        cap.set(1, frame_n)
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
    
    # Close video
    cap.release()
    return frames

In [ ]:
#frames = extract_frames_from_video('../../data/video_1.mp4')
frames = extract_frames_from_video(env_vars["URL_CAMERA_TIENDA"])

In [ ]:
raw_img = frames[0]

In [ ]:
plot_image(raw_img)

### Crop mostrador step by step

In [ ]:
vitr_pts = {'top-left': (840, 430),
            'top-right': (320, 1380),
            'bottom-left': (970, 510),
            'bottom-right': (390, 1510)}

In [ ]:
img_pts = raw_img.copy()
for y, x in vitr_pts.values():
    # Radius of circle 
    radius = 3
    # Red color in RGB 
    color = (255, 0, 0) 
    # Line thickness of 2 px 
    thickness = 5
    # Using cv2.circle() method 
    img_pts = cv2.circle(img_pts, (x, y), radius, color, thickness) 

In [ ]:
plot_image(img_pts)

In [ ]:
a = vitr_pts['top-right'][1] - vitr_pts['top-left'][1]
b = vitr_pts['top-left'][0] - vitr_pts['top-right'][0]
width = round(sqrt(a**2 + b**2))

In [ ]:
a = vitr_pts['top-left'][1] - vitr_pts['bottom-left'][1]
b = vitr_pts['bottom-left'][0] - vitr_pts['top-left'][0]
height = round(sqrt(a**2 + b**2))

In [ ]:
print(f'Width: {width} - Height: {height}')

In [ ]:
raw_pts = np.float32([[x, y] for y, x in vitr_pts.values()])

In [ ]:
raw_pts

In [ ]:
dst_pts = np.float32([[0, 0], [width, 0],
                      [0, height], [width, height]])

In [ ]:
dst_pts

In [ ]:
M = cv2.getPerspectiveTransform(raw_pts, dst_pts)

In [ ]:
img_crop = cv2.warpPerspective(raw_img, M, (width, height))

In [ ]:
plot_image(img_crop)

In [ ]:
div_lines_x = [240, 500, 770]

In [ ]:
img_lines = img_crop.copy()
for x in div_lines_x:
    # Line coordinates
    start_point = (x, 0) 
    end_point = (x, img_lines.shape[0]) 
    # Red color in RGB
    color = (255, 0, 0) 
    # Line thicknes
    thickness = 2
    # Using cv2.line() method 
    img_lines = cv2.line(img_lines, start_point, end_point, color, thickness) 

In [ ]:
plot_image(img_lines)

In [ ]:
vitr_crops = list()
for idx, x in enumerate(div_lines_x):
    if idx == 0:
        crop = img_crop[: , :x, :]
    elif idx == (len(div_lines_x) - 1):
        crop = img_crop[: , x:, :]
    else:
        crop = img_crop[: , div_lines_x[idx-1]:x]
    vitr_crops.append(crop)

In [ ]:
for vitr_img in vitr_crops:
    plot_image(vitr_img, figsize=(10, 10))

### Wrap up in a function

In [ ]:
def extract_mostrador_imgs(img,
                           vitr_pts = {'top-left': (840, 430),
                                       'top-right': (320, 1380),
                                       'bottom-left': (970, 510),
                                       'bottom-right': (390, 1510)},
                           div_lines_x = [240, 500, 770]):
    # Calculate width - height
    a = vitr_pts['top-right'][1] - vitr_pts['top-left'][1]
    b = vitr_pts['top-left'][0] - vitr_pts['top-right'][0]
    width = round(sqrt(a**2 + b**2))
    
    a = vitr_pts['top-left'][1] - vitr_pts['bottom-left'][1]
    b = vitr_pts['bottom-left'][0] - vitr_pts['top-left'][0]
    height = round(sqrt(a**2 + b**2))
    
    # Crop image with warp perspective
    raw_pts = np.float32([[x, y] for y, x in vitr_pts.values()])
    dst_pts = np.float32([[0, 0], [width, 0],
                          [0, height], [width, height]])
    M = cv2.getPerspectiveTransform(raw_pts, dst_pts)
    img_crop = cv2.warpPerspective(img, M, (width, height))
    
    vitr_crops = list()
    for idx, x in enumerate(div_lines_x):
        if idx == 0:
            crop = img_crop[: , :x, :]
        elif idx == (len(div_lines_x) - 1):
            crop = img_crop[: , x:, :]
        else:
            crop = img_crop[: , div_lines_x[idx-1]:x]
        vitr_crops.append(crop)
    return img_crop, vitr_crops

In [ ]:
for idx, img in enumerate(frames):
    print(f'Frame: {idx}')
    plot_image(img, figsize=(15,15))
    img_crop, _ = extract_mostrador_imgs(img)
    plot_image(img_crop, figsize=(20,20))